<a href="https://colab.research.google.com/github/TiwariPradyumn/PySpark-Basics/blob/main/Welmart_sales_insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark=SparkSession.builder.appName("Welmart Sales Insights").getOrCreate()

In [59]:
csv_file="/content/Welmart Sales Data.csv"
df=spark.read.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.load(csv_file)

In [60]:
df.show(5)

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset 

**Tasks: **

1. Determine the best-selling product sub-category.
2. Identify the product category generating the highest revenue.
3. Compile a top 10 list of the most valuable customers.
4. Determine the state responsible for the highest number of orders.
5. Find the year with the highest revenue generation.




In [6]:
# Determine the best-selling product sub-category.
grouped_df=df.groupBy(["Product Name", "Sub-Category"]).agg(sum("Quantity").alias("Total Quantity"))
grouped_df.show(5)

+--------------------+------------+--------------+
|        Product Name|Sub-Category|Total Quantity|
+--------------------+------------+--------------+
|GBC DocuBind 300 ...|     Binders|          23.0|
|     Tuf-Vin Binders|     Binders|          26.0|
|Logitech B530 USB...|      Phones|          25.0|
|OtterBox Commuter...|      Phones|          16.0|
|Grandstream GXP21...|      Phones|          10.0|
+--------------------+------------+--------------+
only showing top 5 rows



In [7]:
grouped_df.dropna()

DataFrame[Product Name: string, Sub-Category: string, Total Quantity: double]

In [8]:
# cast Total Quantity
grouped_df=grouped_df.withColumn("Total Quantity", col("Total Quantity").cast("integer"))

In [9]:
grouped_df=grouped_df.orderBy("Total Quantity", ascending=False)

In [10]:
grouped_df.show(5)

+--------------------+------------+--------------+
|        Product Name|Sub-Category|Total Quantity|
+--------------------+------------+--------------+
|"Tennsco Stur-D-S...|     Storage|          4384|
|"Tenex 46"" x 60"...| Furnishings|          2267|
|"Rubbermaid Clust...| Furnishings|          2064|
|"Belkin 19"" Vent...|     Storage|          1400|
|Wilson Jones Ledg...|     Binders|          1217|
+--------------------+------------+--------------+
only showing top 5 rows



In [11]:
best_selling_product_sub_category=grouped_df.select("Sub-Category").first()[0]
print(f"The best-selling product sub-category is: {best_selling_product_sub_category}")
grouped_df.select("Sub-Category").limit(1).show()

The best-selling product sub-category is: Storage
+------------+
|Sub-Category|
+------------+
|     Storage|
+------------+



In [13]:
revenue=df.groupBy("Category").agg(sum("Sales").alias("Total Revenue"))
revenue = revenue.withColumn("Total Revenue", col("Total Revenue").cast("decimal(38,3)"))
revenue.show(5)

+---------------+-------------+
|       Category|Total Revenue|
+---------------+-------------+
|Office Supplies|   703502.928|
|      Furniture|   733046.861|
|     Technology|   835900.067|
+---------------+-------------+



In [14]:
revenue.orderBy("Total Revenue", ascending=False).show(1)

+----------+-------------+
|  Category|Total Revenue|
+----------+-------------+
|Technology|   835900.067|
+----------+-------------+
only showing top 1 row



In [18]:
customer=df.groupBy(["Customer ID", "Customer Name"]).agg(sum("Sales").alias("Total Sales Done"))
customer=customer.withColumn("Total Sales Done", col("Total Sales Done").cast("decimal(38,3)"))
customer.show(5)

+-----------+----------------+----------------+
|Customer ID|   Customer Name|Total Sales Done|
+-----------+----------------+----------------+
|   JK-15640|        Jim Kriz|        4752.442|
|   CA-12055| Cathy Armstrong|        1679.724|
|   CD-12280|Christina DeMoss|        1104.184|
|   RB-19435| Richard Bierner|        2663.086|
|   SC-20380|Shahid Collister|        5894.184|
+-----------+----------------+----------------+
only showing top 5 rows



In [19]:
customer.orderBy("Total Sales Done", ascending=False).show(10)

+-----------+------------------+----------------+
|Customer ID|     Customer Name|Total Sales Done|
+-----------+------------------+----------------+
|   SM-20320|       Sean Miller|       25043.050|
|   TC-20980|      Tamara Chand|       19017.848|
|   RB-19360|      Raymond Buch|       15117.339|
|   TA-21385|      Tom Ashbrook|       14595.620|
|   AB-10105|     Adrian Barton|       14355.611|
|   SC-20095|      Sanjit Chand|       14142.334|
|   KL-16645|      Ken Lonsdale|       14071.917|
|   HL-15040|      Hunter Lopez|       12873.298|
|   SE-20110|      Sanjit Engle|       12209.438|
|   CC-12370|Christopher Conant|       12129.072|
+-----------+------------------+----------------+
only showing top 10 rows



In [62]:
df.createOrReplaceTempView("welmart_sales_data")
state=spark.sql("""select State, count(*) as total_orders from welmart_sales_data
group by State order by count(*) desc """)
state.show(1)

+----------+------------+
|     State|total_orders|
+----------+------------+
|California|        2001|
+----------+------------+
only showing top 1 row



In [28]:
state.createOrReplaceTempView("state")
spark.sql("select sum(total_orders) from state").show()

+-----------------+
|sum(total_orders)|
+-----------------+
|             9994|
+-----------------+



In [29]:
df.count()

9994

In [32]:
df1=df.groupBy("State").agg(count("state").alias("total_orders"))
df1.orderBy("total_orders", ascending=False).show(1)

+----------+------------+
|     State|total_orders|
+----------+------------+
|California|        2001|
+----------+------------+
only showing top 1 row



In [33]:
# Find the year with the highest revenue generation.
df.withColumn("Order Year", year(col("Order Date"))).show(5)

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+----------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|Order Year|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+----------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furn

In [63]:
df.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)



In [61]:
df=df.withColumnRenamed("Order Date", "Order_Date")
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order_Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   